In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
#from models.MLP import MLP_classifier
from sklearn.model_selection import train_test_split
from MLP import MLP_classifier
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_X = np.load('/content/drive/MyDrive/skku-ml2023-hw4/train_x.npy')
train_y = np.load('/content/drive/MyDrive/skku-ml2023-hw4/train_y.npy')
valid_X = np.load('/content/drive/MyDrive/skku-ml2023-hw4/valid_x.npy')
valid_y = np.load('/content/drive/MyDrive/skku-ml2023-hw4/valid_y.npy')

test_X = np.load('/content/drive/MyDrive/skku-ml2023-hw4/test_x.npy')

In [4]:
proba_list = []

num_epochs = 80
for i in range(5):
    # Count the occurrences of each class in the training set
    train_counter = Counter(train_y)
    valid_counter = Counter(valid_y)
    # Calculate the target count for each class (1200)
    target_count = 1200

    # Create empty lists to store the balanced data
    balanced_train_X1 = []
    balanced_train_y1 = []

    # Specify the classes to balance
    classes_to_balance1 = [0, 9, 8, 7, 6]

    # Iterate over the classes
    for class_label in train_counter.keys():
        # Check if the current class is one of the classes to balance
        if class_label in classes_to_balance1:
            # Get the samples and labels for the current class
            class_indices = np.where(train_y == class_label)[0]
            class_samples = train_X[class_indices]
            class_labels = train_y[class_indices]

            # Shuffle the samples and labels
            shuffled_indices = np.random.permutation(len(class_indices))
            shuffled_samples = class_samples[shuffled_indices]
            shuffled_labels = class_labels[shuffled_indices]

            # Select the first target_count samples for the current class
            selected_samples = shuffled_samples[:target_count]
            selected_labels = shuffled_labels[:target_count]

            # Add the selected samples and labels to the balanced data
            balanced_train_X1.extend(selected_samples)
            balanced_train_y1.extend(selected_labels)

    # Convert the balanced data lists to numpy arrays
    balanced_train_X1 = np.array(balanced_train_X1)
    balanced_train_y1 = np.array(balanced_train_y1)

    # Check the class distribution of the balanced training data
    print('# of class (balanced train):', Counter(balanced_train_y1))

    # Create empty lists to store the balanced data
    balanced_train_X2 = []
    balanced_train_y2 = []

    # Specify the classes to balance
    classes_to_balance2 = [1,2,3,4,5]

    # Iterate over the classes
    for class_label in train_counter.keys():
        # Check if the current class is one of the classes to balance
        if class_label in classes_to_balance2:
            # Get the samples and labels for the current class
            class_indices = np.where(train_y == class_label)[0]
            class_samples = train_X[class_indices]
            class_labels = train_y[class_indices]

            # Shuffle the samples and labels
            shuffled_indices = np.random.permutation(len(class_indices))
            shuffled_samples = class_samples[shuffled_indices]
            shuffled_labels = class_labels[shuffled_indices]

            # If there are fewer samples than the target, duplicate them
            if len(shuffled_samples) < target_count:
                factor = target_count // len(shuffled_samples)
                remainder = target_count % len(shuffled_samples)
                shuffled_samples = np.concatenate([shuffled_samples]*factor + [shuffled_samples[:remainder]])
                shuffled_labels = np.concatenate([shuffled_labels]*factor + [shuffled_labels[:remainder]])
        
            # Select the first target_count samples for the current class
            selected_samples = shuffled_samples[:target_count]
            selected_labels = shuffled_labels[:target_count]

            # Add the selected samples and labels to the balanced data
            balanced_train_X2.extend(selected_samples)
            balanced_train_y2.extend(selected_labels)

    # Convert the balanced data lists to numpy arrays
    balanced_train_X2 = np.array(balanced_train_X2)
    balanced_train_y2 = np.array(balanced_train_y2)

    # Check the class distribution of the balanced training data
    print('# of class (balanced train):', Counter(balanced_train_y2))

    # 합치기 전에 형식이 맞는지 확인하세요. (예: balanced_train_X1.shape == balanced_train_X2.shape)
    assert balanced_train_X1.shape[1:] == balanced_train_X2.shape[1:], "Train X shapes do not match"
    assert balanced_train_y1.shape[1:] == balanced_train_y2.shape[1:], "Train y shapes do not match"

    # X 배열 합치기
    balanced_train_X = np.concatenate((balanced_train_X1, balanced_train_X2), axis=0)

    # y 배열 합치기
    balanced_train_y = np.concatenate((balanced_train_y1, balanced_train_y2), axis=0)

    train_images, train_labels = balanced_train_X, balanced_train_y

    valid_images, valid_labels = valid_X, valid_y

    # Preprocess the dataset (normalize and convert to tensors)
    test_images = test_X
    train_images, valid_images = train_images / 255.0, valid_images / 255.0
    test_images = test_images / 255.0

    train_images = torch.tensor(train_images, dtype=torch.float32).reshape(-1, 28*28)
    train_labels = torch.tensor(train_labels, dtype=torch.long)
    valid_images = torch.tensor(valid_images, dtype=torch.float32).reshape(-1, 28*28)
    valid_labels = torch.tensor(valid_labels, dtype=torch.long)
    test_images = torch.Tensor(test_images).reshape(-1, 28*28)

    # Create DataLoader for training and testing sets
    train_dataset = TensorDataset(train_images, train_labels)
    valid_dataset = TensorDataset(valid_images, valid_labels)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)

    # Create the model
    input_size = 28*28
    num_classes = 10
    model = MLP_classifier(input_size, num_classes)

    # Set up the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model using the train method
    model.train(train_loader, valid_loader, criterion, optimizer, num_epochs)
    proba_list.append(model.predict_proba(test_images))

# of class (balanced train): Counter({0: 1200, 6: 1200, 7: 1200, 8: 1200, 9: 1200})
# of class (balanced train): Counter({1: 1200, 2: 1200, 3: 1200, 4: 1200, 5: 1200})
Epoch [1/80], Loss: 1.6097
Accuracy: 75.60%
Epoch [2/80], Loss: 1.6342
Accuracy: 77.20%
Epoch [3/80], Loss: 1.6247
Accuracy: 76.60%
Epoch [4/80], Loss: 1.6353
Accuracy: 80.20%
Epoch [5/80], Loss: 1.6299
Accuracy: 81.40%
Epoch [6/80], Loss: 1.6234
Accuracy: 81.80%
Epoch [7/80], Loss: 1.6217
Accuracy: 80.40%
Epoch [8/80], Loss: 1.6084
Accuracy: 82.50%
Epoch [9/80], Loss: 1.6926
Accuracy: 81.30%
Epoch [10/80], Loss: 1.5752
Accuracy: 80.80%
Epoch [11/80], Loss: 1.5733
Accuracy: 82.00%
Epoch [12/80], Loss: 1.6458
Accuracy: 78.70%
Epoch [13/80], Loss: 1.5648
Accuracy: 82.80%
Epoch [14/80], Loss: 1.5447
Accuracy: 81.50%
Epoch [15/80], Loss: 1.5536
Accuracy: 81.40%
Epoch [16/80], Loss: 1.5599
Accuracy: 83.10%
Epoch [17/80], Loss: 1.5111
Accuracy: 80.90%
Epoch [18/80], Loss: 1.5662
Accuracy: 83.50%
Epoch [19/80], Loss: 1.5839
Acc

In [5]:
results = torch.zeros_like(proba_list[0])
for proba in proba_list:
    results += proba

In [6]:
predicted = results.argmax(-1)

In [7]:
result = pd.DataFrame({"index":np.arange(len(predicted)), "label": predicted})
result.to_csv("result_ensemble.csv", index=None)